In [ ]:
# ITNOA
%%capture
try:
  import hazm
  print('Alll packages are installed!')
except:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  !pip install torchaudio
  !pip install librosa
  !pip install jiwer
  !pip install hazm
  !pip install num2fawords

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import evaluate
import librosa
import torch
import torchaudio

import pandas as pd
import numpy as np

import hazm
from num2fawords import words, ordinal_words
from tqdm import tqdm

from sklearn.model_selection import train_test_split

import os
import string
import six
import re
import glob
import IPython.display as ipd

In [ ]:
_normalizer = hazm.Normalizer()

chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬",'ٔ', ",", "?",
    ".", "!", "-", ";", ":",'"',"“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š',
     #"ء",
]

# In case of farsi
chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)

chars_to_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",

    # "ها": "  ها", "ئ": "ی",

    "a": " ای ", "b": " بی ", "c": " سی ", "d": " دی ", "e": " ایی ", "f": " اف ",
    "g": " جی ", "h": " اچ ", "i": " آی ", "j": " جی ", "k": " کی ", "l": " ال ",
    "m": " ام ", "n": " ان ", "o": " او ", "p": " پی ", "q": " کیو ", "r": " آر ",
    "s": " اس ", "t": " تی ", "u": " یو ", "v": " وی ", "w": " دبلیو ", "x": " اکس ",
    "y": " وای ", "z": " زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",
}


def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))

def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + " "
    return text

def text_normalizer(text, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = text.lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)

    text = " ".join(_text) + " "
    text = text.strip()

    if not len(text) > 0:
        return None

    return text + " "

def batch_normalizer(batch, chars_to_ignore=chars_to_ignore, chars_to_mapping=chars_to_mapping):
    chars_to_ignore_regex = f"""[{"".join(chars_to_ignore)}]"""
    text = batch["sentence"].lower().strip()

    text = _normalizer.normalize(text)
    text = multiple_replace(text, chars_to_mapping)
    text = remove_special_characters(text, chars_to_ignore_regex)
    text = re.sub(" +", " ", text)
    _text = []
    for word in text.split():
        try:
            word = int(word)
            _text.append(words(word))
        except:
            _text.append(word)

    text = " ".join(_text) + " "
    text = text.strip()

    batch["sentence"] = text + " "
    return batch

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr = sampling_rate, target_sr = 16_000)

    batch["speech"] = speech_array
    return batch

def predict(batch):
    features = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    pred_ids = torch.argmax(logits, dim=-1)

    batch["predicted"] = processor.batch_decode(pred_ids)[0]
    return batch

In [ ]:
#@title Kaggle Credential

import json

username = "" #@param {type: "string"}
api_key = "" #@param {type: "string"}


if username and api_key:
    token = {"username": username, "key": api_key}

    !mkdir ~/.kaggle
    !mkdir /content/.kaggle
    with open('/content/.kaggle/kaggle.json', 'w') as f:
        json.dump(token, f)

    !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
    !chmod 600 /root/.kaggle/kaggle.json

    print('Your are ready to use kaggle API!')


Your are ready to use kaggle API!


In [ ]:
%%capture
import os
if not os.path.exists('/content/stuff/shemo'):
  !mkdir -p /content/stuff/shemo
  !kaggle datasets download mansourehk/shemo-persian-speech-emotion-detection-database
  !unzip shemo-persian-speech-emotion-detection-database.zip -d /content/stuff/shemo/
  !wget -O stuff/shemo/shemo.json https://github.com/pariajm/sharif-emotional-speech-dataset/raw/master/shemo.json

In [ ]:
abs_path_to_data = os.path.join("/content/stuff", "shemo")
!ls {abs_path_to_data}

female	male  shemo.json  transcript


In [ ]:
import json
shemo_json_file = open(abs_path_to_data + '/shemo.json')
shemo_json = json.load(shemo_json_file)
shemo_json_file.close()

In [ ]:
from tqdm import tqdm
data = []
for _id, val in tqdm(shemo_json.items()):
    male_path = f"{abs_path_to_data}/male/{_id}.wav"
    female_path = f"{abs_path_to_data}/female/{_id}.wav"
    text = val['transcript']
    if "M" in _id and os.path.exists(male_path):
        data.append({
            "_id": _id,
            "sentence": text,
            "path": male_path.strip(),
            "gender": "M"
        })
    elif "F" in _id and os.path.exists(female_path):
        data.append({
            "_id": _id,
            "sentence": text,
            "path": female_path.strip(),
            "gender": "F"
        })

100%|██████████| 3000/3000 [00:00<00:00, 162036.08it/s]


In [ ]:
import pandas as pd
from IPython.display import display, HTML

df = pd.DataFrame(data)
df_samples = pd.DataFrame(data[:10])
display(HTML(df_samples.to_html()))

,_id,sentence,path,gender
0,M36S02,﻿عشق هر دوی ما رو به مرز بی‌قراری و جنون کشونده بود,/content/stuff/shemo/male/M36S02.wav,M
1,F21N23,جلوی پیرهن اینجا است. این پاپیون مال پشت پیرهنته.,/content/stuff/shemo/female/F21N23.wav,F
2,M21F01,اون مال جوونیامه,/content/stuff/shemo/male/M21F01.wav,M
3,M28A43,و مقام و منسب دوکی را از خانوادۀ ما گرفتن,/content/stuff/shemo/male/M28A43.wav,M
4,F26N02,می‌دونین که این هفته نوبت کشیک شب شوهرمه.,/content/stuff/shemo/female/F26N02.wav,F
5,F24H15,اون هم ببینه اینقدر این کار رو دوست داری باهات همکاری می‌کنه.,/content/stuff/shemo/female/F24H15.wav,F
6,F24H01,بیمار دچار نوعی بیماری قلبی پیشرفته در قسمت دریچه‎های قلب است.,/content/stuff/shemo/female/F24H01.wav,F
7,M02N14,زندگی ایشون در این مدت بیم بود و امید,/content/stuff/shemo/male/M02N14.wav,M
8,F24A54,اصلاً تو کی هستی؟,/content/stuff/shemo/female/F24A54.wav,F
9,M28H02,با این حال خیال می‏کنم نتیجۀ نقشۀ تو این می‎شه که فردا من رو با خواری و خفت از اینجا بیرون کنن,/content/stuff/shemo/male/M28H02.wav,M


In [ ]:
print(f"Step 0: {len(df)}")

'''df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status", 1)
print(f"Step 1: {len(df)}")

# df["prev_sentence"] = df["sentence"]
df["sentence"] = df["sentence"].apply(lambda t: text_normalizer(t))  # Normalizing!
df = df.dropna(subset=["sentence"])
print(f"Step 2: {len(df)}")'''

# df = df[["prev_sentence", "sentence", "path"]]
# df = df[["sentence", "path"]]
#df = df.sample(frac=1)
#df = df.reset_index(drop=True)
#df.head()

df["sentence"] = df["sentence"].apply(lambda t: text_normalizer(t))  # Normalizing!
df_samples = df[:30]
display(HTML(df_samples.to_html()))

Step 0: 2838


,_id,sentence,path,gender
0,M36S02,عشق هر دوی ما رو به مرز بی قراری و جنون کشونده بود,/content/stuff/shemo/male/M36S02.wav,M
1,F21N23,جلوی پیرهن اینجا است این پاپیون مال پشت پیرهنته,/content/stuff/shemo/female/F21N23.wav,F
2,M21F01,اون مال جوونیامه,/content/stuff/shemo/male/M21F01.wav,M
3,M28A43,و مقام و منسب دوکی را از خانواده ما گرفتن,/content/stuff/shemo/male/M28A43.wav,M
4,F26N02,می دونین که این هفته نوبت کشیک شب شوهرمه,/content/stuff/shemo/female/F26N02.wav,F
5,F24H15,اون هم ببینه اینقدر این کار رو دوست داری باهات همکاری می کنه,/content/stuff/shemo/female/F24H15.wav,F
6,F24H01,بیمار دچار نوعی بیماری قلبی پیشرفته در قسمت دریچه های قلب است,/content/stuff/shemo/female/F24H01.wav,F
7,M02N14,زندگی ایشون در این مدت بیم بود و امید,/content/stuff/shemo/male/M02N14.wav,M
8,F24A54,اصلا تو کی هستی,/content/stuff/shemo/female/F24A54.wav,F
9,M28H02,با این حال خیال می کنم نتیجه نقشه تو این می شه که فردا من رو با خواری و خفت از اینجا بیرون کنن,/content/stuff/shemo/male/M28H02.wav,M


In [ ]:
import torchaudio
import librosa
import IPython.display as IPythonDisplay
import numpy as np

idx = np.random.randint(0, len(df))
print(idx)
sample = df.iloc[idx]

path = sample["path"]
print(sample["sentence"], "\n")
speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()

speech = librosa.resample(np.asarray(speech), orig_sr = sr, target_sr = 16_000)
IPythonDisplay.Audio(data=np.asarray(speech), autoplay=False, rate=16000)

2121
می بخشید که بی اجازه شما این کارو کردم  



In [ ]:
train_df, test_df = train_test_split(df, test_size = 0.1, random_state = 313, stratify = df["gender"])

train_df = train_df[["path", "sentence"]]
train_df = train_df.reset_index(drop=True)

test_df = test_df[["path", "sentence"]]
test_df = test_df.reset_index(drop=True)

print(train_df.shape)
print(test_df.shape)
print(train_df['sentence'][0])
print(test_df['sentence'][0])

(2554, 2)
(284, 2)
چون کویی کوئک روزه بود و فکر می کردم که روزه اش تا غروب آفتاب طول می کشه تصمیم گرفتم که تا غروب در شهر گردش کنم 
از طرفی شنیدیم که امیررضا شریفی جانباز ورزشکار دارای کارت اهدای عضو هنگام خروج از باشگاه دچار سانحه می شه و جان می بازه 


In [ ]:
print(abs_path_to_data)
train_df.to_csv(f"{abs_path_to_data}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{abs_path_to_data}/test.csv", sep="\t", encoding="utf-8", index=False)

/content/stuff/shemo


In [ ]:
shemo_train = load_dataset("csv", data_files={"train": abs_path_to_data + "/train.csv"}, delimiter="\t")["train"]
shemo_test = load_dataset("csv", data_files={"test": abs_path_to_data + "/test.csv"}, delimiter="\t")["test"]

print(shemo_train)
print(shemo_test)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'sentence'],
    num_rows: 2554
})
Dataset({
    features: ['path', 'sentence'],
    num_rows: 284
})


In [ ]:
shemo_train = shemo_train.map(speech_file_to_array_fn, num_proc=4)#, batched=True)
shemo_test = shemo_test.map(speech_file_to_array_fn, num_proc=4)#, batched=True)

Map (num_proc=4):   0%|          | 0/2554 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/284 [00:00<?, ? examples/s]

In [ ]:
shemo_train

Dataset({
    features: ['path', 'sentence', 'speech'],
    num_rows: 2554
})

In [ ]:
shemo_test

Dataset({
    features: ['path', 'sentence', 'speech'],
    num_rows: 284
})

In [ ]:
#model_name_or_path = "m3hrdadfi/wav2vec2-large-xlsr-persian-shemo"  # to test
model_name_or_path = "lnxdx/B1_1000_1e-4_hp-mehrdad"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForCTC.from_pretrained(model_name_or_path).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [ ]:
test_result = shemo_test.map(predict)
train_result = shemo_train.map(predict)

Parameter 'function'=<function predict at 0x7e7e19d97e20> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/284 [00:00<?, ? examples/s]

Map:   0%|          | 0/2554 [00:00<?, ? examples/s]

In [ ]:
#result.save_to_disk('123')

In [ ]:
max_items = np.random.randint(0, len(test_result), 100).tolist()
for i in max_items:
    reference, predicted =  test_result["sentence"][i], test_result["predicted"][i]
    print("reference:", reference)
    print("predicted:", predicted)
    print('---')

reference: می خواستم ببینم شما به چه حق به خودتون اجازه دادید که این مباشر رو به ما تحمیل کنید 
predicted: می خواستم ببینم شما به چه حق به خودتون اجازه دادید که این مباشه را به ما تحمیل کنید
---
reference: تمام دیشبو منتظرت بودم 
predicted: تمام دیشب و منتظرت بودم
---
reference: شده که برای سرچارلز نامه بنویسید و از اون بخواید که به دیدنتون بیاد 
predicted: شده که برایسرچارلز نامه بنویسید و از اون بخواید که به دیدنتون بیاد
---
reference: من دیگه نمی تونم یه زن موفق باشم 
predicted: من دیگه نمی تونم یه زن م وفق باشم
---
reference: منم می خوام از اون نفرت بزرگ فرار کنم می فهمی 
predicted: منم می خوام از اون نسرت بزرگ فرار کنم می فهمی
---
reference: مادمازل مارتن هم همین الان می رسه 
predicted: ماید بزل مارتون هم همین الان می رسیم
---
reference: مزخرفه مزخرف 
predicted: مزخرف مزخرف
---
reference: تمام سوالات منو در یک کلمه جواب بده آره یا نه 
predicted: تمام سوالات منو در یک کلمه جواب بده آره یا نه
---
reference: می خواستم ببینم شما به چه حق به خودتون اجازه دادید که این مباشر رو به ما تحم

In [ ]:
wer_metric = evaluate.load('wer')
wer = wer_metric.compute(predictions=test_result["predicted"], references=test_result["sentence"])
print(f"Test WER: {wer * 100 : .2f}")

Test WER:  28.94


In [ ]:
wer_metric = evaluate.load('wer')
wer = wer_metric.compute(predictions=train_result["predicted"], references=train_result["sentence"])
print(f"Train WER: {wer * 100 : .2f}")

Train WER:  15.66


### Testing on Common Voice 13.0


In [ ]:
dataset = load_dataset("mozilla-foundation/common_voice_13_0", name = "fa")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 9405it [00:00, 94045.61it/s]
Reading metadata...: 28024it [00:00, 116081.57it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 10440it [00:00, 111131.08it/s]


...


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 10440it [00:00, 88642.01it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 20673it [00:00, 109161.65it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 14071it [00:00, 111648.05it/s]


In [ ]:
test_dataset = dataset['test']
test_dataset

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
    num_rows: 10440
})

In [ ]:
test_dataset = test_dataset.map(
    batch_normalizer,
    fn_kwargs={"chars_to_ignore": chars_to_ignore, "chars_to_mapping": chars_to_mapping},
    remove_columns=list(set(test_dataset.column_names) - set(['sentence', 'path']))
)

Map:   0%|          | 0/10440 [00:00<?, ? examples/s]

In [ ]:
test_dataset = test_dataset.map(speech_file_to_array_fn)

Map:   0%|          | 0/10440 [00:00<?, ? examples/s]

In [ ]:
test_dataset

Dataset({
    features: ['path', 'sentence', 'speech'],
    num_rows: 10440
})

In [ ]:
test_result = test_dataset.map(predict)

Map:   0%|          | 0/10440 [00:00<?, ? examples/s]

In [ ]:
max_items = np.random.randint(0, len(test_result), 20).tolist()
for i in max_items:
    reference, predicted =  test_result["sentence"][i], test_result["predicted"][i]
    #print("reference:", reference)
    #print("predicted:", predicted)
    #print('---')

In [ ]:
wer_metric = evaluate.load('wer')
common_voice_wer = wer_metric.compute(predictions=test_result["predicted"], references=test_result["sentence"])
print(f"Common Voice Test WER: {common_voice_wer * 100 : .2f}")

Common Voice Test WER:  19.30
